In [1]:
# 필요한 모듈 임포트
import pandas as pd
import tensorflow as tf
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, Conv1D, GlobalMaxPool1D, concatenate
import os

In [2]:
# ① 데이터 읽어오기
train_file = os.path.join('./models/intent', "Intent_train_data.csv")
df = pd.read_csv(train_file, delimiter=',')
df

,"네, 다른 사이드 메뉴는 없나요?",1,메뉴판 요구
0,빵 종류가 어떻게 되죠?,1,메뉴판 요구
1,샌드위치 판매하나요?,1,메뉴판 요구
2,빵 종류는 진열된 게 전부인가요?,1,메뉴판 요구
3,메뉴판 보여주실 수 있나요?,1,메뉴판 요구
4,메뉴판 좀 부탁드려요,1,메뉴판 요구
...,...,...,...
104638,아메리카노 테이크 아웃으로 해주세요,10,NaN
104639,네 테이크아웃 할인돼요?,10,NaN
104640,테이크아웃이랑 홀에서 먹는 거랑 가격이 다른가요?,10,NaN
104641,테이크아웃은 더 싼가요?,10,NaN


In [3]:
df.columns = ['query', 'intent', 'intent_info']
seq_len = list(map(lambda x : len(x.split(' ')), df['query']))
max(seq_len)  # 문장 최대길이

14

In [4]:
# 단어 시퀀스 벡터 크기
MAX_SEQ_LEN = max(seq_len)

def GlobalParams():
    global MAX_SEQ_LEN

In [5]:
# '질문(query)' 과 '의도(intent)'
queries = df['query'].tolist()
intents = df['intent'].tolist()

In [6]:
queries[:20]

['빵 종류가 어떻게 되죠?',
 '샌드위치 판매하나요?',
 '빵 종류는 진열된 게 전부인가요?',
 '메뉴판 보여주실 수 있나요?',
 '메뉴판 좀 부탁드려요',
 '메뉴판 이거밖에 없어요?',
 '메뉴판 좀 볼 수 있을까요?',
 '브레드 종류는 뭐가 있나요?',
 '플렛 화이트 아이스도 있나요?',
 '커피 메뉴 말고는 없나요?',
 '커피 메뉴는 어디 있어요?',
 '커피종류는 어떤 거 있어요',
 '차 종류는 어디에 있어요?',
 '아 그럼 겨울에만 판매되는 메뉴는 있나요?',
 '메뉴 다 되는 건가요?',
 '빵 종류는 뭐가 있나요?',
 '가나슈 가능 한가요',
 '갈릭 치즈 브레드 가능 한가요',
 '감귤라테 가능 한가요',
 '거품시나몬가루 가능 한가요']

In [7]:
# 전처리 모듈 불러오기
from utils.Preprocess import Preprocess
p = Preprocess(word2index_dic=os.path.join('./train_tools/dict', 'chatbot_dict.bin'),
               userdic=os.path.join('./utils', 'my_dict.tsv'))

In [8]:
# 단어 시퀀스 생성 (가장 시간 많이 걸림)
# 해당 단어에 매칭되는 번호로 시퀀스 생성

# ★ 시간 제법 걸림 * 몇십초 정도..

sequences = []
for sentence in queries:
    pos = p.pos(sentence)
    keywords = p.get_keywords(pos, without_tag=True)
    seq = p.get_wordidx_sequence(keywords)
    sequences.append(seq)

In [ ]:
# ② 단어 인덱스 시퀀스 벡터 
# 단어 시퀀스 벡터 크기 (MAX_SEQ_LEN 로 동일하게 맞추기, 패딩처리)
from config.GlobalParams import MAX_SEQ_LEN
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')

In [ ]:
# ③ 학습용, 검증용, 테스트용 데이터셋 생성
ds = tf.data.Dataset.from_tensor_slices((padded_seqs, intents)) # 패딩처리된 시퀀스와 의도(intent) 리스트 전체를 데이터셋 객체로
ds = ds.shuffle(len(queries)) # 랜덤 섞기

# 학습셋:검증셋:테스트셋 = 7:2:1
train_size = int(len(padded_seqs) * 0.7)
val_size = int(len(padded_seqs) * 0.2)
test_size = int(len(padded_seqs) * 0.1)

train_ds = ds.take(train_size).batch(20)
val_ds = ds.skip(train_size).take(val_size).batch(20)
test_ds = ds.skip(train_size + val_size).take(test_size).batch(20)

# 하이퍼 파라미터 설정
dropout_prob = 0.5
EMB_SIZE = 128
EPOCH = 5
VOCAB_SIZE = len(p.word_index) + 1 #전체 단어 개수


In [ ]:
# ④ CNN 모델 정의
# keras 함수형 모델 방식으로 구현
input_layer = Input(shape=(MAX_SEQ_LEN,))  # 입려크기
embedding_layer = Embedding(VOCAB_SIZE, EMB_SIZE, input_length=MAX_SEQ_LEN)(input_layer)
dropout_emb = Dropout(rate=dropout_prob)(embedding_layer)

conv1 = Conv1D(
    filters=128,
    kernel_size=3,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool1 = GlobalMaxPool1D()(conv1)

conv2 = Conv1D(
    filters=128,
    kernel_size=4,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool2 = GlobalMaxPool1D()(conv2)

conv3 = Conv1D(
    filters=128,
    kernel_size=5,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool3 = GlobalMaxPool1D()(conv3)

# 3,4,5gram 이후 합치기
concat = concatenate([pool1, pool2, pool3])

hidden = Dense(128, activation=tf.nn.relu)(concat)
dropout_hidden = Dropout(rate=dropout_prob)(hidden)
logits = Dense(5, name='logits')(dropout_hidden)  # 최종적으로 5가의 의도 클래스를 분류. 결과로 나온 값(logits) 을을 점수(score) 라 부른다
predictions = Dense(5, activation=tf.nn.softmax)(logits)


In [ ]:
# ⑤ 모델 생성 
model = Model(inputs=input_layer, outputs=predictions)
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
# 모델학습
# ★ 시간 걸림 ★
model.fit(train_ds, validation_data=val_ds, epochs=EPOCH, verbose=1)

In [22]:
# ⑦ 모델 평가(테스트 데이터 셋 이용)
loss, accuracy = model.evaluate(test_ds, verbose=1)
print('Accuracy: %f' % (accuracy * 100))
print('loss: %f' % (loss))

529/529 [==============================] - 1s 1ms/step - loss: 0.0056 - accuracy: 0.9974
Accuracy: 99.744439
loss: 0.005575


In [23]:
# ⑧ 모델 저장
model.save(os.path.join('./models/intent','intent_model.h5'))

## 의도분류 모듈 테스트

In [25]:
from utils.Preprocess import Preprocess
from models.intent.IntentModel import IntentModel
import os

p = Preprocess(word2index_dic=os.path.join('./train_tools/dict', 'chatbot_dict.bin'),
               userdic=os.path.join('./utils', 'user_dic.tsv'))

In [26]:
intent = IntentModel(model_name=os.path.join('./models/intent', 'intent_model.h5'), preprocess=p)

In [27]:
query = "아메리카노 주세요"

In [28]:
predict = intent.predict_class(query)
predict_label = intent.labels[predict]

In [29]:
print(query)
print('의도 예측 클래스 : ', predict)
print('의도 예측 레이블 : ', predict_label)

오늘 탕수육 주문 가능한가요?
의도 예측 클래스 :  2
의도 예측 레이블 :  주문
